In [2]:
pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp313-cp313-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   -------

  You can safely remove it manually.
  You can safely remove it manually.


In [3]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import random
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense,BatchNormalization, Flatten, MaxPool2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from keras.layers import Conv2D, Reshape
from tensorflow.keras.utils import Sequence
from keras.backend import epsilon
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam


from sklearn.metrics import classification_report, confusion_matrix

import os

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [4]:
train_dir = "D:/DOG AND CAT/train1"
train_class=os.listdir("D:/DOG AND CAT/train1")
print(train_class)


test_dir = "D:/DOG AND CAT/test1"
test_class=os.listdir(test_dir)
print(test_class)

['cats', 'dogs']
['cats', 'dogs']


In [ ]:
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

x = []  # image arrays
y = []  # labels

for folder in os.listdir("D:/DOG AND CAT/train1"):
    folder_path = os.path.join(train_dir, folder)
    if not os.path.isdir(folder_path):  # skip if not a folder
        continue
    
    image_list = os.listdir(folder_path)
    for img_name in image_list:
        img_path = os.path.join(folder_path, img_name)

        # ✅ Skip if it's not a file (e.g., a folder or hidden file)
        if not os.path.isfile(img_path):
            continue
        
        try:
            # Load and preprocess image
            img = image.load_img(img_path, target_size=(224, 224))
            img = image.img_to_array(img)
            img = preprocess_input(img)

            x.append(img)
            y.append(train_class.index(folder))

        except Exception as e:
            print(f"Skipping {img_path}, error: {e}")

print("Preparing Training Dataset Completed.")

In [ ]:
#Code by 'Transfer Learning - Image Classification + Deploy' https://www.kaggle.com/code/yash161101/transfer-learning-image-classification-deploy
# Training Dataset
print("Training Dataset")

x=np.array(x) # Converting to np arrary to pass to the model
print(x.shape)

y=to_categorical(y) # onehot encoding of the labels
# print(y)
print(y.shape)

print("Test Dataset")

test_images=np.array(test_images) 
print(test_images.shape)

test_image_label=to_categorical(test_image_label) # onehot encoding of the labels)
print(test_image_label.shape)



In [ ]:
xtrain, xval, ytrain, yval = train_test_split(x,y,test_size=0.2,random_state=5)
print("Splitting data for train and val completed.")


In [ ]:
import os

print("Train dir:", train_dir)
for root, dirs, files in os.walk(train_dir):
    print(root, "->", len(files), "files")

print("\nTest dir:", test_dir)
for root, dirs, files in os.walk(test_dir):
    print(root, "->", len(files), "files")

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(x,y,test_size=0.2,random_state=5)
print("Splitting data for train and val completed.")

In [ ]:
rand_images1 = random.sample(os.listdir("D:/DOG AND CAT/train1/dogs"), 8)
rand_images2 = random.sample(os.listdir("D:/DOG AND CAT/train1/cats"), 8)

In [ ]:
import cv2
fig = plt.figure(figsize=(10,10))
for i, im in enumerate(rand_images1):
    plt.subplot(2,4 , i+1)
    im = cv2.imread(os.path.join("D:/DOG AND CAT/train1/dogs", im))
    plt.imshow(im)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
for i, im in enumerate(rand_images2):
    plt.subplot(2,4 , i+1)
    im = cv2.imread(os.path.join("D:/DOG AND CAT/train1/cats", im))
    plt.imshow(im)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
mnet = MobileNetV2(include_top = False, weights = "imagenet" ,input_shape=(224,224,3))

In [ ]:
tf.keras.backend.clear_session()

model = Sequential([
    mnet,
    GlobalAveragePooling2D(),
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation="relu"),
    Dropout(0.1),
    Dense(32, activation="relu"),
    Dropout(0.3),
    Dense(2, activation="sigmoid")
])

model.layers[0].trainable = False

model.compile(
    loss="categorical_crossentropy", 
    optimizer="Adam", 
    metrics=["accuracy"]
)

model.summary()

In [ ]:
def scheduler(epoch):
    if epoch <= 2:
        return 0.001
    elif epoch > 2 and epoch <= 15:
        return 0.0001 
    else:
        return 0.00001

lr_callbacks = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
EPOCHS = 7
hist = model.fit(xtrain, ytrain, epochs=EPOCHS, callbacks=[lr_callbacks], validation_data=(xval,yval))